# Adding BxD dataset (Rachel Meade, Clare Smith) to standardized datafiles: 

In [1]:
import pandas as pd
import os

# INTERLUDE: 
### Checking out the number and identity of screens in the dash website database (updated as of 07/10/23)

We want to make sure both are equally up to date. 

In [ ]:
path_dash = '/home/ajinich/Documents/repos/mtb_tn_db_demo/data'
fn_path = os.path.join(path_dash, 'standardized_data_dash.tsv')
df_dash = pd.read_csv(fn_path, sep = '\t')
list_exp = df_dash.Expt.unique().tolist()

In [ ]:
print('there are this many total screens in the dash site:', len(list_exp))

In [ ]:
list_CC = [exp for exp in list_exp if 'CC' in exp]
list_CC = list_CC + ['129s1.SvImJ_vs_in_vitro',
    'A.J_vs_in_vitro', 
    'Cast.EiJ_vs_in_vitro',
    'IFNG_vs_in_vitro',
    'NOD.ShiLtJ_vs_in_vitro',
    'NOS_vs_in_vitro',
    'NZO.H1LtJ_vs_in_vitro',
    'PWK.PhJ_vs_in_vitro',
    'Phox_vs_in_vitro',
    'Rag_vs_in_vitro',
    'Wsb.Eij_vs_in_vitro',
    'mbio_H37Rv_vs_in_vitro',
    'C57BL.6J_vs_in_vitro']
print('There are this many CC-panel screens in the dash site:', len(list_CC))

In [ ]:
list_bxd = [exp for exp in list_exp if 'BXD' in exp]
list_bxd = list_bxd + ['C57BL.6J_vs_in_vitro_H37Rv', 'DBA.2J_vs_in_vitro_H37Rv']
print('There are this many BxD screens in the Dash site:', len(list_bxd))

In [ ]:
list_rem = [exp for exp in list_exp if exp not in list_CC + list_bxd]
print('There are this many remainder screens:', len(list_rem))

In [ ]:
list_carey = [exp for exp in list_rem if 'carey' in exp]
list_KO = [exp for exp in list_rem if 'KO' in exp]
list_dejesus = [exp for exp in list_rem if 'dejesus' in exp]
list_xu = [exp for exp in list_rem if 'xu_' in exp]
list_zhang = [exp for exp in list_rem if 'zhang' in exp]
list_temp = list_carey + list_KO + list_dejesus + list_xu + list_zhang

list_rem_rem = [exp for exp in list_rem if exp not in list_temp]
list_rem_rem

In [ ]:
len(list_KO)

### Load raw, full BxD datasets from Rachel Meade:

In [ ]:
path_bxd = '../../data/standardized_data/BXD_TnSeq/'
fn_bxd = '2022.02.15_BXD_TRANSIT_BGC_1pseudo_output_RKM_wPvals_Unthresholded.csv'
fn_bxd_path = os.path.join(path_bxd, fn_bxd)

df_bxd = pd.read_csv(fn_bxd_path,  index_col = 0)
df_bxd.head(1)

### Grab BxD L2FC columns: 

In [ ]:
cols_lfc = list(set([col.split('_')[0] for col in df_bxd.columns[5:]]))
cols_lfc.sort()
cols = ['Orf'] + cols_lfc

df_bxd_lfc = df_bxd[cols].copy()
df_bxd_lfc.rename(columns={'Orf':'Rv_ID'}, inplace=True)
df_bxd_lfc.head(1)

### Grab BxD q-value columns: 

In [ ]:
cols_qval = [col for col in df_bxd.columns[5:] if 'Qval' in col]
cols_qval.sort()
cols = ['Orf'] + cols_qval
df_bxd_qval = df_bxd[cols].copy()
df_bxd_qval.rename(columns={'Orf':'Rv_ID'}, inplace=True)

# Rename columns to get rid of the "Qval"
dict_cols = {col:col.split('_')[0] for col in df_bxd_qval.columns[1:]}
df_bxd_qval.rename(columns=dict_cols, inplace=True)
df_bxd_qval.head(1)

##### Append control condition to column names: 

In [ ]:
dict_cols_lfc = {col:col+'_vs_in_vitro_H37Rv' for col in df_bxd_lfc.columns[1:]}
dict_cols_qval = {col:col+'_vs_in_vitro_H37Rv' for col in df_bxd_qval.columns[1:]}
df_bxd_qval.rename(columns=dict_cols_qval, inplace=True)
df_bxd_lfc.rename(columns=dict_cols_lfc, inplace=True)

### Grab standardized dataset (L2FC dataframe, q-values dataframe)

#### PAUSE! 
- Make sure you grab the correct datasets. 
- Use this as an opportunity to clean up the data/standardized_data folder. Send everything else to an old dataset.
- Document where you added the CC-panel datasets. 

In [ ]:
path_std = '../../data/standardized_data/old_std_data/'
fn_std_lfc = os.path.join(path_std, 'result_logfc_matrix_2023_02_21_CC_invitroref.csv')
fn_std_qval = os.path.join(path_std, 'result_qval_matrix_2023_02_21_CC_invitroref.csv')

# Load as dataframes: 
df_std_lfc = pd.read_csv(fn_std_lfc)
df_std_qval = pd.read_csv(fn_std_qval)
# shapes: 
print(df_std_lfc.shape, df_std_qval.shape)

#### Merging LFC dataframes: 
print(df_bxd_lfc.shape, df_std_lfc.shape)
df_std_bxd_lfc = df_std_lfc.merge(df_bxd_lfc, how = 'left', on='Rv_ID')
print(df_std_bxd_lfc.shape)

#### Merging q-value dataframes: 
print( df_bxd_qval.shape, df_std_qval.shape )
df_std_bxd_qval = df_std_qval.merge(df_bxd_qval, how = 'left', on='Rv_ID')
print( df_std_bxd_qval.shape )

### Write to files: 
fn_std_bxd_qval = os.path.join(path_std, 'result_qval_matrix_2023_02_20_CC_BxD.csv')
fn_std_bxd_lfc = os.path.join(path_std, 'result_logfc_matrix_2023_02_20_CC_BxD.csv')
df_std_bxd_qval.to_csv(fn_std_bxd_qval, index = False)
df_std_bxd_lfc.to_csv(fn_std_bxd_lfc, index = False)

#### Repeat but with data that has both in-vitro and mBio controls: 

In [ ]:
path_std = '../../data/standardized_data/old_std_data/'
fn_std_lfc = os.path.join(path_std, 'result_logfc_matrix_2023_02_21_CC_invitro_mbio_ref.csv')
fn_std_qval = os.path.join(path_std, 'result_qval_matrix_2023_02_21_CC_invitro_mbio_ref.csv')

# Load as dataframes: 
df_std_lfc = pd.read_csv(fn_std_lfc)
df_std_qval = pd.read_csv(fn_std_qval)
# shapes: 
print(df_std_lfc.shape, df_std_qval.shape)

#### Merging LFC dataframes: 
print(df_bxd_lfc.shape, df_std_lfc.shape)
df_std_bxd_lfc = df_std_lfc.merge(df_bxd_lfc, how = 'left', on='Rv_ID')
print(df_std_bxd_lfc.shape)

#### Merging q-value dataframes: 
print( df_bxd_qval.shape, df_std_qval.shape )
df_std_bxd_qval = df_std_qval.merge(df_bxd_qval, how = 'left', on='Rv_ID')
print( df_std_bxd_qval.shape )

### Write to files: 
fn_std_bxd_lfc = os.path.join(path_std, 'result_logfc_matrix_2023_02_20_CC_w_mbio_BxD.csv')
fn_std_bxd_qval = os.path.join(path_std, 'result_qval_matrix_2023_02_20_CC_w_mbio_wBxD.csv')
df_std_bxd_lfc.to_csv(fn_std_bxd_lfc, index = False)
df_std_bxd_qval.to_csv(fn_std_bxd_qval, index = False)

## Make sure everything is in order in the SI datasets: 

In [ ]:
path_SI = '../../data/SI_datasets/'
fn_SI = os.path.join(path_SI, 'SI_log2FC.csv')
df_lfc_SI = pd.read_csv(fn_SI)
df_lfc_SI.shape

In [ ]:
#